In [2]:
import re
import json
import numpy as np

In [14]:
month2number = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12'
}

reg_exprs = [
    re.compile(r'от \d?\d [А-Яа-я]+ \d\d\d\d'),
    re.compile(r'\d\d\.\d\d\.\d\d\d\d'),
    re.compile(r'\n\d?\d [А-Яа-я]+ \d\d\d\d')
]

DAY = re.compile(r'\s\d?\d ')
MONTH = re.compile(r' [А-Яа-я]+ ')
YEAR = re.compile(r'\d\d\d\d')

In [15]:
def extract_date(doc):
    date = ''
    matches = []
    for expr in reg_exprs:
        matches += expr.findall(doc)
    if matches:
        if matches[0].startswith('от') or matches[0].startswith('\n'):
            day = DAY.search(matches[0]).group().strip()
            if len(day) == 1:
                day = '0' + day
            month = MONTH.search(matches[0]).group().strip()
            if month in month2number:
                month = month2number[month]
            else:
                month = '00'
            year = YEAR.search(matches[0]).group()
            date = f'{day}.{month}.{year}'
        else:
            date = matches[0]
    return date

def predict(test):
    results = []
    for doc in test:
        date = extract_date(doc)
        prediction = {"type": "",
                      "date": date,
                      "number": "",
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

In [16]:
from eval_module import quality

train = []
labeled = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

predicted = predict(train)
quality(predicted, labeled)

{'date_accuracy': 0.6818751567,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 0}

In [17]:
predicted

[{'type': '', 'date': '06.03.2006', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '31.12.2014', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '23.06.2014', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '15.08.1996', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '31.12.2014', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '27.11.2010', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '31.12.1996', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '25.12.2012', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '23.07.2013', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '23.07.2013', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '21.02.1992', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '16.10.2012', 'number': '', 'authority': '', 'name': ''},
 {'type': '', 'date': '30.12.2012', 'num